<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">

# Accelerating End-to-End Data Science Workflows # 

## 07 - Triton ##

**Table of Contents**
<br>
This notebook covers the process of deploying a model with Triton Inference Server and walks through the various tools available within Triton. This notebook covers the below sections: 
1. [Background](#s1)
2. [Preparing the Model](#s1)
    * [Load the Model](#s1)
    * [Creating the Folder Structure](#s1)
    * [Creating the Configuration File](#s1)
3. [Loading the Model in Triton](#s1)
    * [Starting Triton](#s1)
    * [Check Status of Triton Server](#s1)
4. [Testing Inference](#s1)
    * [Triton Client](#s1)
    * [Verifying Results for Local Model and Triton](#s1)
5. [Analyzing Performance](#s1)
    * [Customizing Perf Analyzer](#s1)
    * [Exercise #1 - Testing Perf Analyzer](#s1)
6. [Model Analyzer](#s1)


# Background
NVIDIA offers a framework for deploying ML models called Triton. Triton will automatically handle all inference requests that comes to the server. Triton supports multiple backends including PyTorch, TensorFlow, Forest Inference Library (FIL), etc. In this notebook, we will focus on the FIL backend using the xgboost model that was trained previously. 

## Preparing the Model
### Load the Model
Let's start with loading the previous XGBoost model.

In [ ]:
import xgboost as xgb
model = xgb.Booster({'nthread': 4})  # init model
model.load_model('xgboost_model.json')  # load model data

### Creating the Folder Structure
In the previous notebook, we just saved the XGBoost model to the working directory. Triton expects the model to be in a particular structure: Model name should be the top level directory, and the version number should be next. This allows for multiple models and versions of those models to be hosted simulatenously. Let's create that folder structure and save the model!

In [ ]:
import os

# Create the model repository directory. The name of this directory is arbitrary.
REPO_PATH = os.path.abspath('models')
os.makedirs(REPO_PATH, exist_ok=True)

# The name of the model directory determines the name of the model as reported by Triton
model_dir = os.path.join(REPO_PATH, "virus_prediction")

# We can store multiple versions of the model in the same directory. In our case, we have just one version, so we will add a single directory, named '1'.
version_dir = os.path.join(model_dir, '1')
os.makedirs(version_dir, exist_ok=True)

# The default filename for XGBoost models saved in json format is 'xgboost.json'.
# It is recommended that you use this filename to avoid having to specify a name in the configuration file.
model_file = os.path.join(version_dir, 'xgboost.json')
model.save_model(model_file)

### Creating the Configuration File
Triton also requires a configuration file that provides details about the model and deployment. For this notebook, we will assume default parameters.

In [ ]:
config_text = f"""backend: "fil"
max_batch_size: {32768}
input [                                 
 {{  
    name: "input__0"
    data_type: TYPE_FP32
    dims: [ 4 ]                    
  }} 
]
output [
 {{
    name: "output__0"
    data_type: TYPE_FP32
    dims: [ 1 ]
  }}
]
instance_group [{{ kind: KIND_GPU }}]
parameters [
  {{
    key: "model_type"
    value: {{ string_value: "xgboost_json" }}
  }},
  {{
    key: "output_class"
    value: {{ string_value: "false" }}
  }},
  {{
    key: "storage_type"
    value: {{ string_value: "AUTO" }}
  }}
]

dynamic_batching {{
  max_queue_delay_microseconds: 100
}}"""
config_path = os.path.join(model_dir, 'config.pbtxt')
with open(config_path, 'w') as file_:
    file_.write(config_text)

Now the model is ready to be loaded in Triton! The model repository should look like this.

```
model/
`-- virus_prediction
    |-- 1
    |   `-- xgboost.model
    `-- config.pbtxt
```


## Loading the Model in Triton
Next, we will need to start the Triton server. For this course, the server has already been started, but we will briefly go the steps required.

### Starting Triton
Triton is available as buildable source code or a pre-built docker image. For simplicity, we recommend that most users start with the docker images. This is how you would start the docker container in the console.

In [ ]:
#!docker run --gpus=1 --rm -p 8000:8000 -p 8001:8001 -p 8002:8002 -v /full/path/to/docs/examples/model_repository:/models nvcr.io/nvidia/tritonserver:<xx.yy>-py3 tritonserver --model-repository=/models

Wow! That's a lot of inputs. Let's break it down a little bit more.

- **gpus=1** : Passes through the first GPU to the Triton Inference Server
- **rm** : Removes the container after completing execution
- **p 8000:8000** : Forwards the GTPCInferenceService Port
- **p 8001:8001** : Forwards the HTTPService Port
- **p 8002:8002** :  Forwards the Metrics Port
- **v /full/path/to/docs/examples/model_repository:/models** : Mounts the path to the models folder on the host machine to the Triton Inference Server container
- **nvcr.io/nvidia/tritonserver:<xx.yy>-py3** : Name of the Triton Inference Server image. The version number will change depending on the latest release
- **tritonserver --model-repository=/models**: The command to run in the container. In this case, we start the Triton Inference Server and point to the models folder


As we mentioned before, the server has already been started for this lab. Let's check our connection to the server! We are using "triton" as the hostname because the default docker network will resolve "triton" to the ip address of the Triton Inference Server.

In [ ]:
!curl -v triton:8000/v2/health/ready

Now let's see if the model has been loaded!

In [ ]:
!curl -X POST http://triton:8000/v2/repository/index

If all went well, we should be able to see the model "virus prediction" show up as ready.

## Testing Inference

### Triton Client
To test the deployment we will use the Triton Client library. Let's see how we can create an instance of the client. 

In [ ]:
import time
import tritonclient.grpc as triton_grpc
from tritonclient import utils as triton_utils
HOST = "triton"
PORT = 8001
TIMEOUT = 60

In [ ]:
client = triton_grpc.InferenceServerClient(url=f'{HOST}:{PORT}')

Now let's make sure the Triton server is ready by sending a sample inference request. First let's load the training data. We will only load 32768 rows, as that's the max batch size we specified.

In [ ]:
import cudf 
import numpy as np
df = cudf.read_csv('./data/clean_uk_pop_full.csv', usecols=['age', 'sex', 'northing', 'easting', 'infected'], nrows=5000000)
df = df.sample(32768)
input_data = df.drop('infected', axis=1)
target = df[['infected']]
print(target)

Now we convert it to a numpy array and force the type to be float32 (the same type we specified in the config file)

In [ ]:
converted_df = input_data.to_numpy(dtype='float32')

Since we limited our batch size to 32768, let's splice the array and attempt the inference.

In [ ]:
%%time
batched_data = converted_df[:32768]
# Prepare the input tensor
input_tensor = triton_grpc.InferInput("input__0", batched_data.shape, 'FP32')
input_tensor.set_data_from_numpy(batched_data)

# Prepare the output
output = triton_grpc.InferRequestedOutput("output__0")

# Send inference request
response = client.infer("virus_prediction", [input_tensor], outputs=[output])

# Get the output data
output_data = response.as_numpy("output__0")

Let's make sure the results we got were the same as with the local model.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

xgb_data = xgb.DMatrix(input_data)
y_test = model.predict(xgb_data)

In [ ]:
# Check that we got the same accuracy as previously
#target = target.to_numpy()
import matplotlib.pyplot as plt

false_pos_rate, true_pos_rate, thresholds = roc_curve(target.to_numpy(), y_test)
auc_result = auc(false_pos_rate, true_pos_rate)

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(false_pos_rate, true_pos_rate, lw=3,
        label='AUC = {:.2f}'.format(auc_result))
ax.plot([0, 1], [0, 1], 'k--', lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend(loc='lower right');
plt.show()


In [ ]:
# Check that we got the same accuracy as previously
#target = target.to_numpy()
import matplotlib.pyplot as plt

false_pos_rate, true_pos_rate, thresholds = roc_curve(target.to_numpy(), output_data)
auc_result = auc(false_pos_rate, true_pos_rate)

fig, ax = plt.subplots(figsize=(5, 5))
ax.plot(false_pos_rate, true_pos_rate, lw=3,
        label='AUC = {:.2f}'.format(auc_result))
ax.plot([0, 1], [0, 1], 'k--', lw=2)
ax.set(
    xlim=(0, 1),
    ylim=(0, 1),
    title="ROC Curve",
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
)
ax.legend(loc='lower right');
plt.show()

As we can see, our AUC score is the same between both inference options!

## Analyzing Performance

Earlier, we tested a *relatively* small inference request. What if we want to see the max throughput of the model? Luckily, Triton offers a performance analysis tool that generates synthetic data to collect latency and throughput numbers. Let's try it out.

In [ ]:
!perf_analyzer -m virus_prediction -u "triton:8000"

That's a lot of information to take in. Let's break it down.

**Measurement window**: Timeframe that measurements are taken in 

**Batch Size**: Number of inputs in each request


**Concurrency**: Number of simulatenous connections
**Latency**: Time taken to recieve results
**p50/90/95/99**: Different percentiles for latency

Based on these results, we can see that our throughput is roughly ~2300 inferences per second with a single concurrent connection, and the average latency for each requst is 434 usec.

# Customizing Perf Analyzer
The Performance Analyzer tool for Triton has many knobs that can be turned to analyze results. Let's enable GPU metric collection, and increase the batch size and concurrency range values!

In [ ]:
!perf_analyzer --collect-metrics -m virus_prediction -u "triton:8000" -b 8 --concurrency-range 2:8:2

The results show that our model configuration gives a throughput of about ~156171 inferences per second. Notice how there are significant throughput gains as we increase the number of concurrent connections. With low concurrency values, Triton is idle during the time when the response is returned to the client and the next request is received at the server. Throughput increases when we increase concurrency values because Triton overlaps the processing of one request with the communication of the other. 

# Exercise 
Please take this time to experiment with the perf analyzer tool. A full list of parameters can be seen with the --help argument. Some parameters that we recommend trying out are listed below. 

-b <value> : batch size
--concurrency-range <start:end:step> : range of concurrency values to test
--collect-metrics : enable collection of GPU metrics

## Model Analyzer
Although it is out of scope for this course, we would like to introduce the Model Analyzer tool that is available as part of Triton. This tool searches through different parameter configurations to find optimal parameters that maximize inference throughput. With some minor processing, results can be viewed in a PDF format as well. The syntax for running Model Analyzer is shown below, as well as an example of the output. Model Analyz

In [ ]:
#%%bash
## Writing constraints to file
#cat > model_analyzer_constraints.yaml <<EOL 
#model_repository: /model_repository/
#triton_launch_mode: "local"
#latency_budget: 5
#run_config_search_max_concurrency: 64
#run_config_search_max_instance_count: 3
#run_config_search_max_preferred_batch_size: 8
#profile_models:
#  virus_prediction
#
#EOL

In [ ]:
# Run model_analyzer profiler on XGBoost model 
#!model-analyzer profile -f model_analyzer_constraints.yaml --override-output-model-repository

![image](images/model_analyzer.png)

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**Well Done!** Let's move to the [next notebook](3-08_k-means_dask.ipynb). 

<img src="images/nvidia_header.png" style="margin-left: -30px; width: 300px; float: left;">